# NJ→NY crossings, by mode
From [the NYMTC's "Hub-Bound Travel" reports](https://www.nymtc.org/en-us/Data-and-Modeling/Transportation-Data-and-Statistics/Publications/Hub-Bound-Travel) about modes+volume of trips to and from Manhattan's Central Business District, 2014-2021.

Reports are mirrored in [this GDrive folder](https://drive.google.com/drive/folders/16YYlcHoCA3scyvCNXfBKEf0P_41IRpJS).

In [1]:
from utz import *

In [2]:
xls_paths = {
    2014: '2014/DM_TDS_Hub_Bound_Travel_2014/DM_TDS_Hub_Bound_Travel_Quick Reference Table_2014.xlsx',
    2015: '2015/DM_TDS_Hub_Bound_Travel_2015/DM_TDS_Hub_Bound_Travel_Quick Reference Table_2015.xlsx',
    2016: '2016/2016 Hub Bound - Excel files/DM_TDS_Hub_Bound_Travel_Quick Reference Table_2016.xlsx',
    2017: '2017/HUBBOUND2017/DM_TDS_Hub_Bound_Travel_Quick Reference Table_2017.xlsx',
    2018: '2018/2018hubboundexcelfiles/DM_TDS_Hub_Bound_Travel_Quick Reference Table_2018.xlsx',
    2019: '2019/2019_Hub_Bound_Travel_Excel_Tables/DM_TDS_Hub_Bound_Travel_Quick Reference Table_2019.xlsx',
    2020: '2020/2020_Hub_Bound_Excel_tables./DM_TDS_Hub_Bound_Travel_Quick Reference Table_2020.xlsx',
    2021: '2021/2021_HubBound_Excel/DM_TDS_Hub_Bound_Travel_Quick Reference Table_2021.xlsx',
}

def load_nj_day_modes(year):
    path = xls_paths[year]
    qr = read_excel(path)
    qr.columns = list(range(len(qr.columns)))

    if year == 2016:
        qr = qr[[ 0, 1, 3, 5 ]]
    elif year < 2016:
        pass
    else:
        qr = qr[[1, 2, 4, 6]]

    qr.columns = ['Mode'] + qr.loc[5].tolist()[1:]

    [start] = qr[qr.Mode == 'NEW JERSEY'].index.tolist()
    [end] = qr[qr.Mode == 'TOTAL  NEW JERSEY SECTOR - ALL MODES'].index.tolist()

    qr = qr.iloc[start+1:end].set_index('Mode')
    qr = qr.replace('-', nan).dropna(how='all')
    qr['Year'] = year
    qr = qr.reset_index().set_index(['Year', 'Mode']).astype(int)
    return qr

In [3]:
qrs = pd.concat([
    load_nj_day_modes(year)
    for year in range(2014, 2022)
])
qrs

Entering  Leaving   Total
Year Mode                                                    
2014 SUBWAY and PATH                  111310   111590  222900
     AUTOS, TAXIS, VANS AND TRUCKS    111455   123182  234637
     BUS                              192349   196954  389303
     SUBURBAN AND INTERCITY RAIL       90623    91129  181752
     FERRY                             15233    13836   29069
2015 SUBWAY and PATH                  122182   123355  245537
     AUTOS, TAXIS, VANS AND TRUCKS    116797   131852  248649
     BUS                              214609   241565  456174
     SUBURBAN AND INTERCITY RAIL       97090    97287  194377
     FERRY                             16997    17890   34887
2016 SUBWAY and PATH                  128822   124921  253743
     AUTOS, TAXIS, VANS AND TRUCKS    112890   126253  239143
     BUS                              215478   215629  431107
     SUBURBAN AND INTERCITY RAIL      105413    95392  200805
     FERRY                             18264    17572   35836
2017 SUBWAY and PATH                  136898   132736  269634
     AUTOS, TAXIS, VANS AND TRUCKS    115008   124817  239825
     BUS                              219336   215171  434507
     SUBURBAN AND INTERCITY RAIL      100102    97297  197399
     FERRY                             19731    16170   35901
2018 SUBWAY and PATH                  137310   132635  269945
     AUTOS, TAXIS, VANS AND TRUCKS    111688   124127  235815
     BUS                              217485   207487  424972
     SUBURBAN AND INTERCITY RAIL      101072    95668  196740
     FERRY                             19876    16464   36340
2019 SUBWAY and PATH                  140020   133427  273447
     AUTOS, TAXIS, VANS AND TRUCKS    111030   123710  234740
     BUS                              212146   196921  409067
     SUBURBAN AND INTERCITY RAIL      107613   104578  212191
     FERRY                             19408    17412   36820
2020 SUBWAY/PATH                       30939    29699   60638
     AUTOS, TAXIS, VANS AND TRUCKS     92103   105844  197947
     BUS                               57543    53793  111336
     SUBURBAN AND INTERCITY RAIL       19808    20350   40158
     FERRY                              3362     3297    6659
2021 SUBWAY/PATH                       61206    58139  119345
     AUTOS, TAXIS, VANS AND TRUCKS    117295   116253  233548
     BUS                               95076    87057  182133
     SUBURBAN AND INTERCITY RAIL       42033    42481   84514
     FERRY                              9238     8599   17837
     ALL TRANSIT                      207553   196276  403829

In [4]:
njs = qrs.Entering.rename('Entries')
njs

Year  Mode                         
2014  SUBWAY and PATH                  111310
      AUTOS, TAXIS, VANS AND TRUCKS    111455
      BUS                              192349
      SUBURBAN AND INTERCITY RAIL       90623
      FERRY                             15233
2015  SUBWAY and PATH                  122182
      AUTOS, TAXIS, VANS AND TRUCKS    116797
      BUS                              214609
      SUBURBAN AND INTERCITY RAIL       97090
      FERRY                             16997
2016  SUBWAY and PATH                  128822
      AUTOS, TAXIS, VANS AND TRUCKS    112890
      BUS                              215478
      SUBURBAN AND INTERCITY RAIL      105413
      FERRY                             18264
2017  SUBWAY and PATH                  136898
      AUTOS, TAXIS, VANS AND TRUCKS    115008
      BUS                              219336
      SUBURBAN AND INTERCITY RAIL      100102
      FERRY                             19731
2018  SUBWAY and PATH                  13731

In [5]:
modes = (
    njs
    .to_frame()
    .reset_index()
    .pivot(index='Year', columns='Mode', values='Entries')
)
modes.loc[~modes['SUBWAY and PATH'].isna(), 'PATH'] = modes['SUBWAY and PATH']
modes.loc[~modes['SUBWAY/PATH'].isna(), 'PATH'] = modes['SUBWAY/PATH']
modes = (
    modes
    .drop(columns=['ALL TRANSIT', 'SUBWAY and PATH', 'SUBWAY/PATH'])
    .astype(int)
    .rename(columns={
        'AUTOS, TAXIS, VANS AND TRUCKS': 'AUTO',
        'SUBURBAN AND INTERCITY RAIL': 'RAIL',
        'FERRY ': 'FERRY'
    })
)
modes = modes[['AUTO', 'BUS', 'PATH', 'RAIL', 'FERRY']]
modes

Mode,AUTO,BUS,PATH,RAIL,FERRY
Year,,,,,
2014,111455,192349,111310,90623,15233
2015,116797,214609,122182,97090,16997
2016,112890,215478,128822,105413,18264
2017,115008,219336,136898,100102,19731
2018,111688,217485,137310,101072,19876
2019,111030,212146,140020,107613,19408
2020,92103,57543,30939,19808,3362
2021,117295,95076,61206,42033,9238


In [6]:
import plotly.express as px

In [7]:
pc = px.colors.qualitative.Plotly
colors = {
    'AUTO': pc[1],
    'BUS': pc[4],
    'PATH': pc[9],
    'RAIL': pc[3],
    'FERRY': pc[0],   
}

In [10]:
fig = px.bar(
    modes,
    labels={ 'value': '# People', 'Mode': '', 'Year': '', },
    color_discrete_map=colors,
).update_layout(
    title=dict(
        text='NJ→NY crossings on a Fall business day',
        x=0.5,
    ),
    legend=dict(
        traceorder='reversed',
    ),
    hovermode='x',
).update_traces(hovertemplate=None)
fig.write_image('nj-ny-day.png', width=900, height=600)
fig

In [11]:
fig = px.bar(
    modes.div(modes.sum(1), axis=0),
    labels={ 'value': '% People', 'Mode': '', 'Year': '', },
    color_discrete_map=colors,
).update_layout(
    title=dict(
        text='NJ→NY crossings on a Fall business day',
        x=0.5,
    ),
    legend=dict(
        traceorder='reversed',
    ),
    hovermode='x',
    yaxis=dict(
        tickformat=',.0%',
        range=[0,1],
    ),
).update_traces(
    hovertemplate=None,
)
fig.write_image('nj-ny-day_pct.png', width=900, height=600)
fig